There is a great deal of silicate vapor (also referred to as gas) in the moon-forming region of a synestia. Any growing silicate moonlets (also referred to as condensates or liquid droplets) will orbit within the moon-forming region of a synestia and will encounter this gas. The gas will impart a drag upon the moonlets, causing them to slow down and lose angular velocity. The orbit of the moonlets will decay over time in accord with the strength of the gas drag acceleration.

In [ ]:
#INSESRT DIAGRAM HERE

Let's get a sense for the magnitude of gas drag acceleration that we're dealing with. It is assumed that the moonlets experience a simple gas drag. The equation is as follows:
$$a_{drag}=\frac{C_D\ A \ (u_{gas} - u_{body})^2}{2 \ m_{body}}$$

Simple gas drag acceleration a$_{drag}$ is dependent on: the mass of the body that is being dragged by the gas $m_{body}$, the cross sectional area of the body that the gas is interacting with A, the gas drag coefficient C$_D$ (what thermal regime we are in), and the relative velocity between the gas and the body being dragged u$_{gas}$-u$_{body}$. Later, we calculate the gas drag coefficient using the Adachi et al (1976) formulation, but here you can play with its value.

In [ ]:
import numpy as np
import math
from ipywidgets import *
v_rel = np.arange(0.0, 300., 1.) #relative velocity between gas and body in m/s

def pltdrag(rad_body, C_D):
    Moon_density = 3300. #lunar density in kg/m3; 3.3 g/cc
    mass_body = Moon_density*(4./3.)*math.pi*(rad_body**3) #mass of body in kg; assume spherical body
    a_drag = 0.5*C_D*math.pi*(rad_body**2)*(v_rel**2)/mass_body #acceleration imparted on body due to gas drag in m/s2; assume spherical cross section
    plt.plot(v_rel, a_drag)
    plt.xlabel('velocity of the gas relative to the velocity of the moonlet (m/s)')
    plt.ylabel('gas drag acceleration (m/s$^2$)')
    plt.show()

#PLOT ACCELERATION RELATIVE TO SOMETHING ELSE (g's, roller coasters, astronauts, etc.)    
#RELATIVE SCALE IN GENERAL: how big the moonlets are (marble, golf ball, grain of sand, yoga ball, soccer field, etc.), how fast the relative and absolute velocity is
#MAYBE THIS SHOULD BE PRINTED OUTPUT?    
style = {'description_width': 'initial'}
interact(pltdrag, rad_body=FloatLogSlider(value=1, base=10, min=-3, max=6, step=1, description='Size of moonlet (m)', style=style),
         C_D=FloatSlider(value=2, min=0, max=2.5, step=0.5, description='Gas drag coefficient C$_D$ (-)', continuous_update=False, readout=True, readout_format='.1f', style=style))

As expected, gas drag acceleration is greater when the mass of the moonlet is smaller and the gas drag coefficient and relative velocity are larger.

Now, let's see how a single moonlet travels through the vapor in a synestia. First, we will look at how a moonlet moves on a stable Keplerian orbit. Next, we will see the effects of the second-order term of gravity on the orbit of the moonlet. Moonlets, once formed, will quickly equilibrate to the midplane. So, we will look at orbits in the midplane first. Moonlets condense as droplets at the photosphere (where the vapor is saturated and has a pressure that lies on the vapor dome). We will check orbits of moonlets condensing at the photosphere (starting off the midplane) next. Let's check the decay time for various sizes of moonlets and various starting semi-major axes.

In [ ]:
#CALCULATE KEPLERIAN ORBIT WITH NORMAL GRAVITY

In [ ]:
#IMPORT GADGET-2 SNAPSHOT (VAPOR IN SYNESTIA)
#FIT THERMAL PARAMETER PROFILES: PRESSURE, TEMPERATURE, DENSITY
#FIT IN-PLANE VELOCITY PROFILE
#CALCULATE ORBITAL DECAY; UPDATE ACCELERATION DUE TO DRAG

In [ ]:
#REFERENCES